In [ ]:
# Train on one dataset and test on the rest
# modules and functions
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest,chi2 
from sklearn import svm 
from sklearn.preprocessing import MinMaxScaler,StandardScaler,MaxAbsScaler,RobustScaler
from sklearn.model_selection import train_test_split,KFold,GridSearchCV,cross_val_score,StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, make_scorer,confusion_matrix
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
import pickle
from sklearn.neural_network import MLPClassifier
from sklearn.utils import shuffle
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.dummy import DummyClassifier
#mport scikitplot as skplt
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE, ADASYN
filenames=[]
samples=[]
for i in os.listdir("Data/"):
    print(i)
    if len(i.strip(".csv"))==10 and i.strip(".csv")!="SRR8670684":
        filenames.append("Data/"+i)
        samples.append(i.strip(".csv"))
filenames
def binary(y):
    y_out=[]
    for i in y:
        if i<0.5:
            y_out.append(0.0)
        else:
            y_out.append(1.0)
    return y_out
def take_data_preprocess(filename,col):
    x=pd.read_csv(filename)
    x=x[x[col]!=-1.0]
    y=np.array(x[col])
    y=binary(y)
    x=x.drop(["Unnamed: 0"],axis=1)
    y=np.array(y).reshape(len(y)  ,)
    y_1=np.where(y==0.0)[0]
    y_2=np.where(y==1.0)[0]
    index=np.random.choice(y_1,len(y_2), replace=False)
    index=np.concatenate((index, y_2), axis=0)
    x_32=x[[ 'AA_x', 'AT_x', 'AG_x', 'AC_x',
       'TA_x', 'TT_x', 'TG_x', 'TC_x', 'GA_x', 'GT_x', 'GG_x', 'GC_x', 'CA_x',
       'CT_x', 'CG_x', 'CC_x', 'AA_y', 'AT_y', 'AG_y', 'AC_y', 'TA_y', 'TT_y',
       'TG_y', 'TC_y', 'GA_y', 'GT_y', 'GG_y', 'GC_y', 'CA_y', 'CT_y', 'CG_y',
       'CC_y']]
    x_32_norm=np.array(normalize(np.array(x_32)))
    x, y = shuffle(x_32_norm[index], y[index], random_state=0)
    return x,y
def take_full_data_preprocess(filename,col):
    x=pd.read_csv(filename)
    x= x.sample(frac=1).reset_index(drop=True)
    x=x[x[col]!=-1.0]
    y=np.array(x[col])
    y=binary(y)
    x=x.drop(["Unnamed: 0"],axis=1)
    y=np.array(y).reshape(len(y)  ,)
    x_32=x[[ 'AA_x', 'AT_x', 'AG_x', 'AC_x',
       'TA_x', 'TT_x', 'TG_x', 'TC_x', 'GA_x', 'GT_x', 'GG_x', 'GC_x', 'CA_x',
       'CT_x', 'CG_x', 'CC_x', 'AA_y', 'AT_y', 'AG_y', 'AC_y', 'TA_y', 'TT_y',
       'TG_y', 'TC_y', 'GA_y', 'GT_y', 'GG_y', 'GC_y', 'CA_y', 'CT_y', 'CG_y',
       'CC_y']]
    x_32_norm=np.array(normalize(np.array(x_32)))
    x, y = x_32_norm,y
    return x,y
def classification_report_with_accuracy_score(y_true, y_pred):
    originalclass.extend(y_true)
    predictedclass.extend(y_pred)
    return accuracy_score(y_true, y_pred) # return accuracy score
def classification_report_with_f1_score(y_true, y_pred):
    originalclass.extend(y_true)
    predictedclass.extend(y_pred)
    return f1_score(y_true, y_pred,average='weighted') # return accuracy score
def confidence_score(y_true,y_pred_prob):

    keys=[0,1]
    dict={}
    i=0
    for key in keys:
        dict[key]=i
        i+=1

    cs=0.0
    for i in range(len(y_true)):
        cs+=y_pred_prob[i][dict[y_true[i]]]


    return cs/len(y_true)
def normalize(x):
    new_x=[]
    
    for i in range(len(x)):
        x_new=[]
        for j in range(16,32):
            a=np.sum(x[i][1:17])
            b=np.sum(x[i][17:33])
            if x[i][j-16]!=0 and b!=0.0:
                x_new.append(x[i][j]*a/( x[i][j-16]*b))
            else:
                x_new.append(x[i][j]*a/(1+ x[i][j-16]*b))
        new_x.append(x_new)
    return new_x
cancer_types=['DANG_PANCREAS',
 'TE11_OESOPHAGUS',
 'SHP77_LUNG',
 'T84_LARGE_INTESTINE',
 'SKMEL30_SKIN',
 'SKHEP1_LIVER',
 'NUGC3_STOMACH',
 'U2OS_BONE',
 'SW579_THYROID']
def plot_roc_curves(model,train_x,train_y,test_x,test_y):
    
    model_dummy = DummyClassifier(strategy='stratified')
    model_dummy.fit(train_x, train_y)
    yhat = model_dummy.predict_proba(test_x)
    naive_probs = yhat[:, 1]


    yhat = model.predict_proba(test_x)
    pos_probs = yhat[:, 1]
    roc_auc = roc_auc_score(test_y, pos_probs)
    fpr, tpr, thresholds = sk.metrics.roc_curve(test_y, pos_probs)
    
    plt.figure(figsize=(7,4))
    plt.plot(fpr, tpr,label="ROC curve of class 1.0 (area = "+str(roc_auc)+")")
    plt.plot([0,1],[0,1],label="",color="black", linestyle='dashed')
    plt.legend()
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC curves")
    plt.show()
for j in range(0,9):
    
    x,y=take_data_preprocess(filenames[j],cancer_types[j])
    
    print(filenames[j],len(x))
    x=StandardScaler().fit_transform(x)
    
    clf = RandomForestClassifier(class_weight='balanced', max_depth=20) 

    trainx,testx,trainy,testy=train_test_split(x,y)
    
    clf.fit(trainx,trainy)
    
    plot_roc_curves(clf,trainx,trainy,testx,testy)
    plt.show()
    importance = clf.feature_importances_
    plt.bar(nid.keys(), importance)
    plt.xlabel("Features")
    plt.ylabel("Feature Importance")
    plt.show()
    #skplt.metrics.plot_roc(testy, clf.predict_proba(testx))
    plt.show()
# train on one sample and test on the rest of the samples

cancer_types=['DANG_PANCREAS',
 'TE11_OESOPHAGUS',
 'SHP77_LUNG',
 'T84_LARGE_INTESTINE',
 'SKMEL30_SKIN',
 'SKHEP1_LIVER',
 'NUGC3_STOMACH',
 'U2OS_BONE',
 'SW579_THYROID']
filenames
filenames
cancer_types = []
for i in filenames:
    data = pd.read_csv(i)
    cancer_types.append(data.columns[4])
cancer_types
accuracy=[]
j1=0
for file_train in  filenames:
    acc=[]
    j2=0
    
    x,y=take_data_preprocess(file_train,cancer_types[j1])
    #print(filenames[j1],len(x))
    x=StandardScaler().fit_transform(x)

    clf = RandomForestClassifier(class_weight='balanced', max_depth=20 , n_estimators=800) 

    clf.fit(x,y)
    
    for file_test in filenames:
        
        print(file_test)

        
        originalclass = []
        predictedclass = []


        x_test,y_test=take_full_data_preprocess(file_test,cancer_types[j2])
        x_test=StandardScaler().fit_transform(x_test)


        originalclass=y_test
        predictedclass=clf.predict(x_test)

        acc.append(accuracy_score(originalclass, predictedclass))

        j2+=1
        

        # print(classification_report(originalclass, predictedclass))
        
        
    accuracy.append(acc)
    j1+=1
accuracy_self=[]
j=0
for filename in filenames:
    
    x,y=take_data_preprocess(filename,cancer_types[j])
    print(filenames[j],len(x))
    x=StandardScaler().fit_transform(x)
    
    clf = RandomForestClassifier(class_weight='balanced', max_depth=20 , n_estimators=800) 
    
    originalclass = []
    predictedclass = []

    outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

    nested_score = cross_val_score(clf, X=x, y=y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))
    
    accuracy_self.append(accuracy_score(originalclass, predictedclass))


    j+=1
    
    print(classification_report(originalclass, predictedclass))
len(accuracy_self),len(accuracy)
for i in range(9):
    print(i)
    accuracy[i][i]=accuracy_self[i]
cancer_types_space = [ "\n".join(i.split("_")) for i in cancer_types]
cancer_types_space
labels = [
    "Thyroid",
    "Oesophagus",
    "Large Intestine",
    "Skin",
    "Liver",
    "Lung",
    "Stomach",
    "Bone",
    "Pancreas"
]
len(accuracy[i]),len(cancer_types_space)
import matplotlib.pyplot as plt

# Create a more compact figure
fig = plt.figure(figsize=(18, 12))  # Reduce figure size slightly

ax = plt.gca()  # Get current axis

for i in range(9):
    plt.scatter(labels, accuracy[i][:9], label=labels[i], color="C" + str(i), s=300)
    plt.scatter(labels[i], accuracy[i][i], color="C" + str(i), s=600, edgecolors='black', linewidth=3)

plt.legend(bbox_to_anchor=(1.0, 1.0), title="Train sample\n", fontsize=30, title_fontsize=50)

plt.xlabel("Test sample", fontsize=50, labelpad=10)
plt.ylabel("Accuracy", fontsize=50, labelpad=10)

plt.xticks(fontsize=25, rotation=60)
plt.yticks(fontsize=40)

# Adjust layout to minimize white space
plt.subplots_adjust(left=0.12, right=0.88, top=0.88, bottom=0.25)

# Save with minimal whitespace
plt.savefig("Figures/Validation.pdf", dpi=500, bbox_inches='tight')

plt.show()
import numpy as np

# Save data
np.savez("Figures/validation_plot_data.npz", 
         cancer_types_space=cancer_types_space, 
         accuracy=accuracy, 
         cancer_types=cancer_types)
# Thresholding
# Load data
data = np.load("Figures/validation_plot_data.npz")

# Access variables
cancer_types_space = data["cancer_types_space"]
accuracy = data["accuracy"]
cancer_types = data["cancer_types"]
def threshold(y_prob,threshold):
    y=[]
    for i in y_prob:
        if i<threshold:
            y.append(0.0)
        else:
            y.append(1.0)
    return y
j1=0
p_threshold=[]
accuracy_final=[]
for file_train in filenames:
    
    
    accuracy=[]
    j2=0
    x,y=take_data_preprocess(file_train,cancer_types[j1])
    print(filenames[0])
    x=StandardScaler().fit_transform(x)

    clf = RandomForestClassifier(class_weight='balanced', max_depth=20 , n_estimators=800) 

    clf.fit(x,y)
    
    p_=[]
    
    for file_test in filenames:
        
        
        if file_test!=file_train:
        
            acc=[]

            originalclass = []
            predictedclass = []


            x_test,y_test=take_full_data_preprocess(file_test,cancer_types[j2])
            x_test=StandardScaler().fit_transform(x_test)


            originalclass=y_test
            predictedprob=clf.predict_proba(x_test)

            p=len(np.where(y_test==0.0)[0])/(len(x_test))

            print(file_test,p)

            p_.append(p)

            for t in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,p]:

                predictedclass=threshold(predictedprob[:,1],t)
                acc.append(accuracy_score(originalclass, predictedclass))

            accuracy.append(acc)
            p_threshold.append(p_)
            
            j2+=1
            
        else:    
            j2+=1
            
    accuracy_final.append(accuracy)
    j1+=1
p=[0.6418389166072701,0.6222270869819031, 0.6539956803455723, 0.652449567723343,0.6458136320651778,0.6151851318253854,0.622280054941083,0.5786644029428409,0.7146575057255882]
import matplotlib.patches as mpatches
import matplotlib.lines as mlines


# Define colors for each cancer type
colors = ["C" + str(i) for i in range(len(cancer_types))]

# Create custom legend
legend_handles = [mlines.Line2D([], [], color=colors[i], marker='o', linestyle='None', markersize=10, label=labels[i]) for i in range(len(cancer_types))]
len(accuracy_final),len(accuracy)
p_threshold
plt.subplots(3,3,figsize=(20,15))
idx=1
for accuracy in accuracy_final:
    j=0
    plt.subplot(3,3,idx)
    plt.title("Trained on "+labels[idx-1],fontsize=15)
    for i in range(8):
        if i == idx-1:
            j+=1
        if cancer_types[j] == "SW579_THYROID":
            print("---")
        plt.scatter([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],accuracy[i][:-1],label=labels[j],color="C"+str(j))
        plt.scatter(p_threshold[j][i],np.array(accuracy)[:,-1][i],s=100,color="C"+str(j),edgecolors="black",linewidth=2)
        j+=1
    plt.xlabel("Threshold",fontsize=15)
    plt.ylabel("Accuracy",fontsize=15)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    idx+=1
plt.legend(handles =  legend_handles , bbox_to_anchor=(1.0, 1.0),fontsize=15,title_fontsize=15,title="Test sample")
plt.tight_layout()

plt.savefig("Figures/thresholding.pdf",dpi=800)

plt.show()
# Save all relevant data
np.savez("Figures/thresholding_plot_data.npz", 
         accuracy_final=accuracy_final, 
         cancer_types=cancer_types, 
         p_threshold=p_threshold, 
         legend_handles=legend_handles)
# Load data
data = np.load("Figures/thresholding_plot_data.npz", allow_pickle=True)

# Extract variables
accuracy_final = data["accuracy_final"]
cancer_types = data["cancer_types"]
p_threshold = data["p_threshold"]
legend_handles = data["legend_handles"]
human_data=pd.read_csv("/project/ReadStatistics/ML_On_Datasets/FinalFile_19datasets_mean.csv")
human_data
# train model on blood data and test on all other data
x,y=take_data_preprocess("/project/ReadStatistics/ML_On_Datasets/FinalFile_20_datasets_mean.csv","categorize")
x=StandardScaler().fit_transform(x)
clf = RandomForestClassifier(class_weight='balanced', max_depth=20 , n_estimators=800)
clf.fit(x,y)


accuracy_final=[]
j1=0
for file_test in filenames:
    accuracy=[]

    x_test,y_test=take_full_data_preprocess(file_test,cancer_types[j1])
    x_test=StandardScaler().fit_transform(x_test)
    originalclass = y_test
    predictedprob=clf.predict_proba(x_test)
    for t in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
        predictedclass=threshold(predictedprob[:,1],t)
        accuracy.append(accuracy_score(originalclass, predictedclass))
    true_threshold=p[j1]
    predictedclass=threshold(predictedprob[:,1],true_threshold)
    accuracy.append(accuracy_score(originalclass, predictedclass))
    j1+=1
    accuracy_final.append(accuracy)
    
p
# plot the results
plt.figure(figsize=(10,15))
idx=0
for accuracy in accuracy_final:
    plt.scatter([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],accuracy[:10],label=labels[idx],color="C"+str(idx),s=100)
    plt.scatter(p[idx],accuracy_final[idx][10],s=100,color="C"+str(idx), edgecolors='black', linewidth=3)
    plt.xlabel("Threshold",fontsize=25)
    plt.ylabel("Accuracy",fontsize=25)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    idx+=1
plt.tight_layout()

legend_handles = [mlines.Line2D([], [], color="C" + str(i), marker='o', linestyle='None', markersize=10, label=labels[i]) for i in range(len(cancer_types))]
legend_handles.append(mlines.Line2D([], [], marker='o', color='black',  
                                    markerfacecolor='white', markersize=10, 
                                    markeredgecolor='black', markeredgewidth=2,
                                    linestyle='None', label="Observed threshold"))


plt.legend(handles =  legend_handles , bbox_to_anchor=(1.0, 1.0),fontsize=15,title_fontsize=15,title="Test sample")

plt.savefig("Figures/thresholding_blood.pdf",dpi=800)
# Save plot data for future use
np.savez("Figures/thresholding_blood_data.npz", 
         accuracy_final=accuracy_final, 
         labels=labels, 
         p=p)

p[j],accuracy_final[j][10]
len(accuracy_final), len(labels)
